<a href="https://colab.research.google.com/github/dohyung-kim/ccri/blob/main/script/pixel/P1_P2_aggregation_pixel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas rasterio numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 50.4 MB/s eta 0:00:00


In [8]:
# Load p2_avg data
df = pd.read_csv(csv_path)

In [10]:
df['P2_arithmetic_avg']

,P2_arithmetic_avg
0,7.945412
1,3.962525
2,0.851215
3,4.846018
4,8.448727
...,...
226,1.128434
227,4.461930
228,5.844139
229,4.381589


In [14]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

# File paths
input_dir = "/content/drive/MyDrive/CCRI_results_misc"
image_folder = "/content/drive/MyDrive/ccri_pixel"
geojson_path = os.path.join(input_dir,"global_bnd_adm0.geojson")
raster_path = os.path.join(input_dir,"p1_avg_pixel.tif")
csv_path = os.path.join(input_dir,"P2_Merged_Normalized_avg.csv")
output_raster_path = os.path.join(input_dir,"p1_p2_avg_pixel.tif")
mask_path = os.path.join(image_folder, "landSeaMask.tif")

# Load country boundaries
gdf = gpd.read_file(geojson_path)

# Load p2_avg data
df = pd.read_csv(csv_path)

# Ensure ISO3 consistency
df["iso3"] = df["iso3"].str.upper()
gdf["iso3"] = gdf["iso3"].str.upper()

# Merge spatial data with p2_avg values
gdf = gdf.merge(df, on="iso3", how="left")

# Open the raster file
with rasterio.open(raster_path) as src:
    profile = src.profile  # Copy metadata
    profile.update(dtype=rasterio.float32)  # Ensure output is float32

    # Initialize output_data with the original raster values
    output_data = src.read(1).astype(np.float32)

    for _, row in gdf.iterrows():
        iso3_code = row["iso3"]
        p2_avg = row["P2_arithmetic_avg"]

        if np.isnan(p2_avg):
            continue  # Skip countries without p2_avg

        geometry = [row["geometry"]]  # Get country polygon

        # Mask the raster using the country's geometry
        out_image, out_transform = rasterio.mask.mask(src, geometry, crop=False, nodata=np.nan)
        out_image = out_image[0]  # Extract the single-band array

        # Compute geometric mean where valid values exist
        valid_mask = ~np.isnan(out_image)
        out_image[valid_mask] = np.sqrt(out_image[valid_mask] * p2_avg)

        # Update only the valid pixels in the output raster
        output_data[valid_mask] = out_image[valid_mask]

with rasterio.open(mask_path) as mask_src:
    landsea_mask = mask_src.read(1)  # Assuming single-band mask
    land_mask = landsea_mask == 1  # Boolean mask for land pixels

scaler = MinMaxScaler(feature_range=(0, 10))
out_image[land_mask] = scaler.fit_transform(out_image[land_mask].reshape(-1, 1)).flatten()

# Save the output raster
with rasterio.open(output_raster_path, 'w', **profile) as dst:
    dst.write(output_data, 1)

print(f"Processed raster saved to {output_raster_path}")


Processed raster saved to /content/drive/MyDrive/CCRI_results_misc/p1_p2_avg_pixel.tif
